In [2]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
#PCA logic flow 

#Step 1: get the standardised features
x = df (with the features from the dataframe)
# Standardizing the features
x = StandardScaler().fit_transform(x)

#Step 2: feeed standardadised features into the PCA
#We can experiment with different PCA n later? 
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ["pc1", "pc2"])

#Step 3: Check if explained variance is good enough
print(pca.explained_variance_ratio_)

#Step 4: 
Visualisation for slides idk 

In [15]:
#read data 
price_data = pd.read_csv(r'C:\Users\ELIZABETH CHENG\source\repos\MLAP\reader\price_pca.csv')
price_df = price_data.set_index("0")

x = price_df
x = StandardScaler().fit_transform(x)
#print(x)

pca = PCA(n_components=50)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = [f"price_component_{i}" for i in range(0, 50)])
display(principalDf)

,price_component_0,price_component_1,price_component_2,price_component_3,price_component_4,price_component_5,price_component_6,price_component_7,price_component_8,price_component_9,...,price_component_40,price_component_41,price_component_42,price_component_43,price_component_44,price_component_45,price_component_46,price_component_47,price_component_48,price_component_49
0,-1.137617,-0.029992,-0.076824,0.013113,0.011546,0.033747,0.002156,0.011527,-0.007085,-0.004462,...,0.001400,-0.000308,0.001637,0.000550,-0.000695,-0.003511,-0.001467,0.000557,0.000900,0.000818
1,-0.973280,-0.030440,-0.004893,0.025489,0.015969,-0.001815,-0.005823,0.002944,-0.002723,-0.003907,...,0.001751,0.003133,-0.000030,-0.000419,0.000824,-0.000422,-0.000780,-0.000607,0.000302,-0.000261
2,-1.058215,0.045797,-0.059806,0.017035,0.023264,0.009505,-0.008048,0.012816,0.001391,-0.009059,...,-0.000693,-0.003160,-0.000703,0.003370,0.002210,0.001975,-0.000737,-0.000358,-0.001416,0.000294
3,-1.238606,0.003186,-0.006590,0.006240,-0.004436,-0.000433,-0.006352,0.005971,0.000427,-0.005249,...,0.001040,0.000276,-0.000403,-0.000371,-0.000130,0.000720,0.000017,0.000241,-0.000301,0.000417
4,-1.208673,-0.017857,0.002696,0.014969,0.001453,-0.003155,-0.007768,0.002928,0.000686,-0.004382,...,0.000983,0.001269,-0.000450,0.000844,0.000099,-0.000274,-0.000166,0.000283,0.000631,-0.000590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,-1.255665,-0.015883,-0.007233,0.002426,-0.003022,0.002527,-0.006470,0.003233,-0.001449,-0.003959,...,-0.001054,0.000240,-0.000404,-0.000215,0.000142,-0.000305,0.000306,0.000201,0.000111,-0.000082
1463,-0.582213,0.172273,0.030810,-0.020441,-0.013323,-0.061221,0.005574,-0.023454,-0.000679,0.020173,...,0.004632,-0.010197,0.005726,0.010499,-0.001235,0.006450,0.003715,-0.003022,-0.003785,-0.003293
1464,-1.153369,-0.016058,-0.007082,-0.005212,-0.001584,0.009888,-0.002952,0.003303,-0.000254,-0.004720,...,-0.000134,0.000747,-0.000069,0.000241,0.000785,0.000413,-0.000088,0.000603,0.000476,-0.000531
1465,0.088343,0.140342,-0.085550,-0.045554,-0.016523,-0.029048,0.011802,0.032332,0.010236,0.096433,...,-0.002419,0.007286,0.002399,-0.001503,0.003394,-0.001935,0.003123,-0.005389,-0.005687,-0.005521
